In [1]:
import d3rlpy
from d3rlpy.algos import COMBO
from sklearn.model_selection import train_test_split
#import gymnasium as gym
import gym
from gym.wrappers import TransformObservation
import numpy as np
import encoders
import os
import json

In [2]:
print(gym.version.VERSION)

0.23.1


## Params

In [3]:
seed = 1
d3rlpy.seed(seed)
use_gpu = True
# prepare environment
#env = gym.make("InvertedPendulum-v2")
#eval_env = gym.make("InvertedPendulum-v2")
env = gym.make("Reacher-v2")
eval_env = gym.make("Reacher-v2")
env.reset(seed=seed)
eval_env.reset(seed=seed)

array([ 9.99997205e-01,  9.95944393e-01,  2.36432274e-03,  8.99709127e-02,
       -7.52674192e-02, -3.06694204e-02,  3.27702594e-03, -9.08008636e-04,
        2.84797318e-01,  4.10616462e-02,  0.00000000e+00])

In [4]:
def observation_edit1(obs):
    new_obs = np.zeros(8)
    new_obs[0] = np.arccos(obs[0])
    new_obs[1] = np.arccos(obs[1])
    new_obs[2:] = obs[4:-1]
    return new_obs

In [5]:
env1 = TransformObservation(env, observation_edit1)
env1.observation_space = gym.spaces.Box(-np.inf, np.inf, shape=(8,), dtype= np.float64 )
print(env1.reset(seed=seed))

eval_env1 = TransformObservation(eval_env, observation_edit1)
eval_env1.observation_space = gym.spaces.Box(-np.inf, np.inf, shape=(8,), dtype= np.float64 )
print(env1.reset(seed=seed))

[ 0.00236432  0.09009274 -0.07526742 -0.03066942  0.00327703 -0.00090801
  0.28479732  0.04106165]
[ 0.00236432  0.09009274 -0.07526742 -0.03066942  0.00327703 -0.00090801
  0.28479732  0.04106165]


## Create dataset

In [11]:
actor_encoder = d3rlpy.models.encoders.DefaultEncoderFactory(dropout_rate=0.2)
# setup algorithm
sac = d3rlpy.algos.SAC(
    batch_size=256,
    actor_encoder_factory=actor_encoder,
    actor_learning_rate=3e-4,
    critic_learning_rate=3e-4,
    temp_learning_rate=3e-4,
    use_gpu=use_gpu
)

# prepare utilities
buffer = d3rlpy.online.buffers.ReplayBuffer(maxlen=1000000, env=env1)

# start training
sac.fit_online(
    env1,
    buffer,
    eval_env=eval_env1,
    n_steps=1000000,
    n_steps_per_epoch=1000,
    update_interval=1,
    update_start_step=1000,
    tensorboard_dir='tensorboard_logs',
    experiment_name='exp_3_SAC_reacher'
)

2023-10-19 16:55:29 [info     ] Directory is created at d3rlpy_logs/exp_3_SAC_reacher_20231019165529
2023-10-19 16:55:29 [debug    ] Building model...
2023-10-19 16:55:29 [debug    ] Model has been built.
2023-10-19 16:55:29 [info     ] Parameters are saved to d3rlpy_logs/exp_3_SAC_reacher_20231019165529/params.json params={'action_scaler': None, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': 0.2}}, 'actor_learning_rate': 0.0003, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'batch_size': 256, 'critic_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'critic_learning_rate': 0.0003, 'critic_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'gamma': 0.99, 'generated_maxlen': 100000, 'initial_temperature': 1.0, 'n_

  0%|          | 0/1000000 [00:00<?, ?it/s]

2023-10-19 16:55:35 [info     ] Model parameters are saved to d3rlpy_logs/exp_3_SAC_reacher_20231019165529/model_1000.pt
2023-10-19 16:55:35 [info     ] exp_3_SAC_reacher_20231019165529: epoch=1 step=1000 epoch=1 metrics={'time_inference': 0.0035159263610839843, 'time_environment_step': 0.0004376025199890137, 'time_step': 0.0040949125289916994, 'rollout_return': -51.400946937955496, 'evaluation': -13.139856928431351} step=1000
2023-10-19 16:55:49 [info     ] Model parameters are saved to d3rlpy_logs/exp_3_SAC_reacher_20231019165529/model_2000.pt
2023-10-19 16:55:49 [info     ] exp_3_SAC_reacher_20231019165529: epoch=2 step=2000 epoch=2 metrics={'time_inference': 0.0012793483734130859, 'time_environment_step': 0.00018240737915039063, 'time_sample_batch': 0.0002991161346435547, 'time_algorithm_update': 0.011489745855331421, 'temp_loss': 2.8656037547588347, 'temp': 0.8666634668707848, 'critic_loss': 0.41049433798342944, 'actor_loss': -1.6416791453957558, 'time_step': 0.013338100910186768,

KeyboardInterrupt: 

In [12]:
# export replay buffer as MDPDataset
dataset = buffer.to_mdp_dataset()

# save MDPDataset
dataset.dump('d3rlpy_data/reacherv2.h5')

## Load the dataset

In [6]:
dataset = d3rlpy.dataset.MDPDataset.load('d3rlpy_data/reacherv2.h5')

In [7]:
train_episodes, test_episodes = train_test_split(dataset, random_state=seed)

## Dynamics learning

In [8]:
def inverted_pendulum_project(x):
    return x[:, 1:]
projection_size = 3

def reacher_project(x):
    return x[:, [1,4,5]]
projection_size = 3

In [7]:
encoder_factory = encoders.SymmetryEncoderFactory(project=inverted_pendulum_project, projection_size=projection_size)
dynamics = d3rlpy.dynamics.ProbabilisticEnsembleDynamics(learning_rate=1e-4, use_gpu=True, state_encoder_factory=encoder_factory)
#dynamics = d3rlpy.dynamics.ProbabilisticEnsembleDynamics(learning_rate=1e-4, use_gpu=True) # Baseline

Using SymmetryEncoderFactory


In [22]:
# same as algorithms
dynamics.fit(train_episodes,
             eval_episodes=test_episodes,
             n_epochs=100,
             scorers={
                'observation_error': d3rlpy.metrics.scorer.dynamics_observation_prediction_error_scorer,
                'reward_error': d3rlpy.metrics.scorer.dynamics_reward_prediction_error_scorer,
                'variance': d3rlpy.metrics.scorer.dynamics_prediction_variance_scorer,
             },
            tensorboard_dir='tensorboard_logs/dynamics',
            experiment_name='SeparateStateAndRewardEncoders')

NameError: name 'dynamics' is not defined

In [12]:
def experiment_dynamics_training(dataset, symmetry_project, projection_size, n_runs, experiment_name, seed=1, use_gpu=True):
    for i in range(n_runs):
        for exp_type in ['default', 'symmetry']:
            # use the same seeds for default and symmetric runs
            train_episodes, test_episodes = train_test_split(dataset, random_state=seed+i)
            if exp_type == 'symmetry':
                state_encoder_factory = encoders.SymmetryEncoderFactory(project=symmetry_project, projection_size=projection_size)
                train_episodes, test_episodes = train_test_split(dataset, random_state=seed+i+1) # remove this later
                dynamics = d3rlpy.dynamics.ProbabilisticEnsembleDynamics(learning_rate=1e-4, use_gpu=use_gpu, state_encoder_factory=state_encoder_factory)
            else:
                dynamics = d3rlpy.dynamics.ProbabilisticEnsembleDynamics(learning_rate=1e-4, use_gpu=use_gpu)
            dynamics.fit(train_episodes,
                 eval_episodes=test_episodes,
                 n_epochs=100,
                 scorers={
                    'observation_error': d3rlpy.metrics.scorer.dynamics_observation_prediction_error_scorer,
                    'reward_error': d3rlpy.metrics.scorer.dynamics_reward_prediction_error_scorer,
                    'variance': d3rlpy.metrics.scorer.dynamics_prediction_variance_scorer,
                 },
                tensorboard_dir='tensorboard_logs/dynamics',
                experiment_name=experiment_name + '_' + exp_type)

In [13]:
experiment_dynamics_training(dataset=dataset, symmetry_project=reacher_project, projection_size=3, n_runs=5, experiment_name="exp_5_dynamics_reacher", use_gpu=True)

2023-10-22 00:05:48 [debug    ] RoundIterator is selected.
2023-10-22 00:05:48 [info     ] Directory is created at d3rlpy_logs/exp_5_dynamics_reacher_default_20231022000548
2023-10-22 00:05:48 [debug    ] Building models...
2023-10-22 00:05:49 [debug    ] Models have been built.
2023-10-22 00:05:49 [info     ] Parameters are saved to d3rlpy_logs/exp_5_dynamics_reacher_default_20231022000548/params.json params={'action_scaler': None, 'batch_size': 100, 'discrete_action': False, 'gamma': 1.0, 'generated_maxlen': 100000, 'learning_rate': 0.0001, 'n_ensembles': 5, 'n_frames': 1, 'n_steps': 1, 'optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0.0001, 'amsgrad': False}, 'real_ratio': 1.0, 'reward_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'reward_scaler': None, 'scaler': None, 'state_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': Fal

Epoch 1/100:   0%|          | 0/4811 [00:00<?, ?it/s]

2023-10-22 00:08:11 [info     ] exp_5_dynamics_reacher_default_20231022000548: epoch=1 step=4811 epoch=1 metrics={'time_sample_batch': 0.000245681202503332, 'time_algorithm_update': 0.018561039167192865, 'loss': 158.64583251195856, 'time_step': 0.0188931956456719, 'observation_error': 1.9341882627994524, 'reward_error': 0.04554997230132077, 'variance': 1.8632635014254653} step=4811
2023-10-22 00:08:11 [info     ] Model parameters are saved to d3rlpy_logs/exp_5_dynamics_reacher_default_20231022000548/model_4811.pt


Epoch 2/100:   0%|          | 0/4811 [00:00<?, ?it/s]

2023-10-22 00:10:32 [info     ] exp_5_dynamics_reacher_default_20231022000548: epoch=2 step=9622 epoch=2 metrics={'time_sample_batch': 0.0002459282439950845, 'time_algorithm_update': 0.018409750858529694, 'loss': -25.502702346984922, 'time_step': 0.01873591287645028, 'observation_error': 1.4926990214166531, 'reward_error': 0.01139987107781643, 'variance': 1.1014125654333635} step=9622
2023-10-22 00:10:32 [info     ] Model parameters are saved to d3rlpy_logs/exp_5_dynamics_reacher_default_20231022000548/model_9622.pt


...

2023-10-22 19:10:24 [info     ] exp_5_dynamics_reacher_symmetry_20231022190327: epoch=3 step=14436 epoch=3 metrics={'time_sample_batch': 0.00025582288764262336, 'time_algorithm_update': 0.017451642804213196, 'loss': -60.73875956731544, 'time_step': 0.01778776637732933, 'observation_error': 0.9950461414000269, 'reward_error': 0.006494370335419913, 'variance': 0.5695077713723158} step=14436
2023-10-22 19:10:24 [info     ] Model parameters are saved to d3rlpy_logs/exp_5_dynamics_reacher_symmetry_20231022190327/model_14436.pt


Epoch 4/100:   0%|          | 0/4812 [00:00<?, ?it/s]

2023-10-22 19:12:42 [info     ] exp_5_dynamics_reacher_symmetry_20231022190327: epoch=4 step=19248 epoch=4 metrics={'time_sample_batch': 0.0002536933221920074, 'time_algorithm_update': 0.0174128503375319, 'loss': -83.1134996138705, 'time_step': 0.017742859108291462, 'observation_error': 0.8791388435914068, 'reward_error': 0.00363882280111391, 'variance': 0.48822060813064266} step=19248
2023-10-22 19:12:42 [info     ] Model parameters are saved to d3rlpy_logs/exp_5_dynamics_reacher_symmetry_20231022190327/model_19248.pt


Epoch 5/100:   0%|          | 0/4812 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Load Dynamics

In [ ]:
# load trained dynamics model
dynamics_model_path = "d3rlpy_logs/ProbabilisticEnsembleDynamics_20231002230632"
dynamics = d3rlpy.dynamics.ProbabilisticEnsembleDynamics.from_json(dynamics_model_path + '/params.json')
dynamics.load_model(dynamics_model_path + '/model_31542.pt')

## Train Offline RL Algorithm

In [ ]:
encoder = d3rlpy.models.encoders.DefaultEncoderFactory(dropout_rate=0.2)
# give COMBO as the generator argument.
combo = COMBO(dynamics=dynamics, critic_encoder_factory=encoder, actor_encoder_factory=encoder,
              use_gpu=use_gpu)

In [ ]:
combo.fit(dataset = train_episodes, eval_episodes=test_episodes, n_steps=100000, n_steps_per_epoch=1000, tensorboard_dir="tensorboard_logs",
         scorers={
            'environment': d3rlpy.metrics.scorer.evaluate_on_environment(eval_env)
        })

In [10]:
def experiment_COMBO_training(dataset, eval_env, experiment_name, save_name, models_dir, symmetry_project, projection_size, seed=1, use_gpu=True):
    model_paths = [filename for filename in os.listdir(models_dir) if filename.startswith(experiment_name+'_dynamics')]
    model_paths = [models_dir + model_paths_i for model_paths_i in model_paths]
    model_paths.sort()
    print(model_paths)

    symmetry_reduced_paths = []
    default_paths = []
    for model_path_i in model_paths:
        f = open(model_path_i +'/params.json')
        model_path_i_params = json.load(f)
        if(model_path_i_params["state_encoder_factory"]['type']=='symmetry'):
            symmetry_reduced_paths.append(model_path_i)
        elif(model_path_i_params["state_encoder_factory"]['type']=='default'):
            default_paths.append(model_path_i)
    print("Default_paths:", default_paths, "Symmetry reduced paths: ", symmetry_reduced_paths)

    # load trained dynamics model
    for i in range(len(default_paths)):
        for type, dynamics_model_path in zip(['symmetry', 'default'],[symmetry_reduced_paths[i], default_paths[i]]):
            # use the same seeds for default and symmetric runs
            train_episodes, test_episodes = train_test_split(dataset, random_state=seed+i)
            if type == 'symmetry':
                state_encoder_factory = encoders.SymmetryEncoderFactory(project=symmetry_project, projection_size=projection_size)
                dynamics = d3rlpy.dynamics.ProbabilisticEnsembleDynamics(learning_rate=1e-4, use_gpu=use_gpu, state_encoder_factory=state_encoder_factory)
                dynamics.build_with_dataset(dataset)
            else:
                dynamics = d3rlpy.dynamics.ProbabilisticEnsembleDynamics.from_json(dynamics_model_path + '/params.json')

            filenames = os.listdir(dynamics_model_path)
            latest_model_path = dynamics_model_path + '/model_' +  str(max([int(filename.strip('model_.pt')) for filename in filenames if filename.endswith(".pt")])) + '.pt'
            dynamics.load_model(latest_model_path)
            print("Loaded model: ", latest_model_path)
            
            encoder = d3rlpy.models.encoders.DefaultEncoderFactory(dropout_rate=0.2)
            # give COMBO as the generator argument.
            combo = COMBO(dynamics=dynamics, critic_encoder_factory=encoder, actor_encoder_factory=encoder, use_gpu=use_gpu)
            combo.fit(dataset = train_episodes, eval_episodes=test_episodes, n_steps=1000000, n_steps_per_epoch=1000,
                      tensorboard_dir="tensorboard_logs",
                     scorers={
                        'environment': d3rlpy.metrics.scorer.evaluate_on_environment(eval_env)
                    },
                     experiment_name=save_name + "_" + type,
                     save_interval=50)


In [ ]:
experiment_COMBO_training(dataset, eval_env1, 'exp_5', save_name='exp_5_COMBO_reacher', models_dir='d3rlpy_logs/', symmetry_project=reacher_project, projection_size=3, seed=1, use_gpu=True)

['d3rlpy_logs/exp_5_dynamics_reacher_default_20231022000548', 'd3rlpy_logs/exp_5_dynamics_reacher_default_20231022074412', 'd3rlpy_logs/exp_5_dynamics_reacher_default_20231022151920', 'd3rlpy_logs/exp_5_dynamics_reacher_symmetry_20231021133449', 'd3rlpy_logs/exp_5_dynamics_reacher_symmetry_20231022035204', 'd3rlpy_logs/exp_5_dynamics_reacher_symmetry_20231022112713']
Default_paths: ['d3rlpy_logs/exp_5_dynamics_reacher_default_20231022000548', 'd3rlpy_logs/exp_5_dynamics_reacher_default_20231022074412', 'd3rlpy_logs/exp_5_dynamics_reacher_default_20231022151920'] Symmetry reduced paths:  ['d3rlpy_logs/exp_5_dynamics_reacher_symmetry_20231021133449', 'd3rlpy_logs/exp_5_dynamics_reacher_symmetry_20231022035204', 'd3rlpy_logs/exp_5_dynamics_reacher_symmetry_20231022112713']
Using SymmetryEncoderFactory
Loaded model:  d3rlpy_logs/exp_5_dynamics_reacher_symmetry_20231021133449/model_481100.pt
2023-10-22 22:05:37 [debug    ] RandomIterator is selected.
2023-10-22 22:05:37 [info     ] Director

Epoch 1/1000:   0%|          | 0/1000 [00:00<?, ?it/s]

2023-10-22 22:05:40 [debug    ] 250000 transitions are generated. fake_transitions=250000 real_transitions=481194
2023-10-22 22:06:06 [info     ] exp_5_COMBO_reacher_symmetry_20231022220537: epoch=1 step=1000 epoch=1 metrics={'time_sample_batch': 0.0010448517799377442, 'time_algorithm_update': 0.02410107946395874, 'critic_loss': 6.164930180072784, 'actor_loss': 0.7840588709590957, 'temp_loss': 1.9782691452503205, 'temp': 0.9607453961968422, 'time_step': 0.025244216203689576, 'environment': -11.378090222772746} step=1000


Epoch 2/1000:   0%|          | 0/1000 [00:00<?, ?it/s]

2023-10-22 22:06:08 [debug    ] 250000 transitions are generated. fake_transitions=500000 real_transitions=481194
2023-10-22 22:06:32 [info     ] exp_5_COMBO_reacher_symmetry_20231022220537: epoch=2 step=2000 epoch=2 metrics={'time_sample_batch': 0.000974318265914917, 'time_algorithm_update': 0.022514952182769776, 'critic_loss': 6.441056068897248, 'actor_loss': -0.3853411850333214, 'temp_loss': 2.197696668386459, 'temp': 0.8796660744547844, 'time_step': 0.023581775665283203, 'environment': -9.45402920163083} step=2000


...

2023-10-22 22:43:55 [debug    ] 250000 transitions are generated. fake_transitions=1250000 real_transitions=481194
2023-10-22 22:44:17 [info     ] exp_5_COMBO_reacher_symmetry_20231022220537: epoch=95 step=95000 epoch=95 metrics={'time_sample_batch': 0.0009264252185821533, 'time_algorithm_update': 0.02084336996078491, 'critic_loss': 178840.828578125, 'actor_loss': 1224.5034395141602, 'temp_loss': -0.7322644879370928, 'temp': 2.5931632697582243, 'time_step': 0.021857815265655518, 'environment': -8.94280079167909} step=95000


Epoch 96/1000:   0%|          | 0/1000 [00:00<?, ?it/s]

2023-10-22 22:44:20 [debug    ] 250000 transitions are generated. fake_transitions=1250000 real_transitions=481194
2023-10-22 22:44:42 [info     ] exp_5_COMBO_reacher_symmetry_20231022220537: epoch=96 step=96000 epoch=96 metrics={'time_sample_batch': 0.0009189109802246094, 'time_algorithm_update': 0.02077023148536682, 'critic_loss': 229219.4055234375, 'actor_loss': 1425.6629993286133, 'temp_loss': -0.7912385091520846, 'temp': 2.88725052189827, 'time_step': 0.02177622604370117, 'environment': -10.061407372358808} step=96000


Epoch 97/1000:   0%|          | 0/1000 [00:00<?, ?it/s]

2023-10-22 22:44:44 [debug    ] 250000 transitions are generated. fake_transitions=1250000 real_transitions=481194


## Load the Policy

In [ ]:
trained_policy = COMBO()
# initialize with dataset
trained_policy.build_with_dataset(dataset)
# Load entire model parameters.
trained_policy.load_model('d3rlpy_logs/COMBO_20230929153035/model_53000.pt')

## See the policy running

In [ ]:
scorer = d3rlpy.metrics.scorer.evaluate_on_environment(eval_env, render=True)
mean_episode_return = scorer(trained_policy)

## Wrapper

In [54]:
import gym
from gym.wrappers import FilterObservation

In [55]:
def observation_edit1(obs):
    new_obs = np.zeros(8)
    new_obs[0] = np.arccos(obs[0])
    new_obs[1] = np.arccos(obs[1])
    new_obs[2:] = obs[4:-1]
    return new_obs

def observation_edit2(obs):
    new_obs = np.array([obs[1], obs[4], obs[5]])
    return new_obs

In [56]:
env0 = gym.make("Reacher-v4")
print(env0.reset(seed=seed))
env1 = TransformObservation(env0, observation_edit1)
env1.observation_space = gym.spaces.Box(-np.inf, np.inf, shape=(8,), dtype= np.float64 )
print(env1.reset(seed=seed))
env2 = TransformObservation(env1, observation_edit2)
env2.observation_space = gym.spaces.Box(-np.inf, np.inf, shape=(3,), dtype= np.float64 )
print(env2.reset(seed=seed))

[ 9.99997205e-01  9.95944393e-01  2.36432274e-03  8.99709127e-02
 -7.52674192e-02 -3.06694204e-02  3.27702594e-03 -9.08008636e-04
  2.84797318e-01  4.10616462e-02  0.00000000e+00]
[ 0.00236432  0.09009274 -0.07526742 -0.03066942  0.00327703 -0.00090801
  0.28479732  0.04106165]
[ 0.09009274  0.00327703 -0.00090801]


/home/yasin/.local/share/virtualenvs/symmetry-cs285-U_vq4NrE/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:69: UserWarning: WARN: Agent's minimum action space value is -infinity. This is probably too low.
  logger.warn(
/home/yasin/.local/share/virtualenvs/symmetry-cs285-U_vq4NrE/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:73: UserWarning: WARN: Agent's maximum action space value is infinity. This is probably too high
  logger.warn(
/home/yasin/.local/share/virtualenvs/symmetry-cs285-U_vq4NrE/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:97: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  logger.warn(
